In [1]:
import sys
import pickle
import pprint
pp = pprint.PrettyPrinter(indent=0)

# Local Modules

### utilities.py

In [2]:
# utilties.py
def get_attributes(pobj):
    """Given any python object returns its data attributes.

    Args:
        pobj (:obj:`Maker`): A python object.

    Returns:
        (:obj:`list` of :obj:`str`): list of attribute names.

    Raises:
        TypeError: Raised if an object type without a __dict__ attribute is given.
    """

    if '__dict__' not in dir(pobj):
        raise TypeError('A Python object with attributes is expected.')

    attrs = {k:v for k,v in pobj.__dict__.items() if not k.startswith('_')}
    return [k for k, v in attrs.items() if type(v) != 'function']


def load_member_profiles(fname = './data/crm_profile.pickle'):
    """The utility loads member profile data.

    Args:
        pobj (:obj:`pickle`): A serialized python dictionary.

    Returns:
        (:obj:`dict` of :obj:`dict`): The python dictionary of profiles.
            The member id is a key, a profile of the member is the value.
        
    """
    with open(fname,'rb') as f:
        pcrm = pickle.load(f)
    
    return pcrm


def load_skill_cooccurance(fname = './data/skill_occurance.pickle'):
    with open(fname, 'rb') as f:
        projects = pickle.load(f)
    return projects

def load_social_media_profile(fname = './data/twitter_profile.pickle'):
    with open(fname, 'rb') as f:
        ptwitter = pickle.load(f)
    return ptwitter

def load_behavioral_profile(fname = None): pass

def load_spirometer_profile(fname = None): pass

def load_values_profile(fname = None): pass

def get_a_dict_item(adict):
    k,v = adict.popitem()
    adict[k] = v
    return {k:v}

def form_network_relations(connections):
    """Given a list of attribute/connection sets, it returns a co-occurance matrix of the attributes
        observed in the sets.
  
    Args:
        pobj (:obj:`list` of :obj:`set` of :obj:`str`): Co-oocurance data.

    Returns:
        (:obj:`dict`): A dictionary which contains a semantic network data where the keys are the nodes (attributes)
        and values are a dictionary which contains the degree and the weighted links of the node.
    """
    attrs = connections
    alltags = set()
    for k in attrs:
        alltags = alltags.union(set(k))
        
    occurance = dict()
    for t in alltags:
        occurance[t] = {'f':0,'co':{}}
        
    for kset in attrs:
        for k in kset:
            occurance[k]['f'] += 1
            for t in kset:
                if t == k: continue
                if t in occurance[k]['co'].keys():
                    occurance[k]['co'][t] += 1
                else:
                    occurance[k]['co'][t] = 1
                    
    return occurance

In [3]:
coMat = load_skill_cooccurance()
net = form_network_relations(coMat)
pp.pprint(net)

{'3d': {'co': {'3dmodel': 1,
            '3dprint': 4,
            'additive': 1,
            'ar': 1,
            'artist': 1,
            'cad': 1,
            'construction': 1,
            'craft': 1,
            'creative': 1,
            'creativity': 1,
            'design': 10,
            'development': 1,
            'digitalfabrication': 3,
            'disability': 1,
            'education': 2,
            'empowering': 1,
            'enabling': 1,
            'engineer': 1,
            'engineering': 2,
            'enterpreneurship': 1,
            'entrepreneur': 2,
            'fablab': 2,
            'health': 1,
            'industry40': 1,
            'innovation': 15,
            'iot': 2,
            'jewellery': 1,
            'lpg': 1,
            'maker': 3,
            'manufacturing': 4,
            'material': 3,
            'metal': 1,
            'open': 2,
            'printer': 1,
            'product': 1,
            'production': 2,
            'proje

'community': {'co': {'colaboration': 1,
                   'computation': 1,
                   'culture': 1,
                   'data': 2,
                   'digitalfabrication': 1,
                   'diy': 1,
                   'empowering': 1,
                   'enabling': 1,
                   'enterpreneurship': 1,
                   'fashion': 1,
                   'housi': 1,
                   'innovation': 1,
                   'integrity': 1,
                   'making': 1,
                   'manufacturing': 1,
                   'network': 1,
                   'open': 1,
                   'opensource': 2,
                   'prototype': 1,
                   'reuse': 1,
                   'robotics': 1,
                   'social': 2,
                   'source': 1,
                   'sustainability': 1},
             'f': 9},
'communitybuilding': {'co': {'socialinnovation': 1, 'systemicdesign': 1},
                     'f': 1},
'composite': {'co': {'aerospace': 1, 'p

                   'economico': 1,
                   'economist': 2,
                   'electronics': 1,
                   'empowering': 2,
                   'energy': 1,
                   'engagement': 1,
                   'engineer': 1,
                   'engineering': 3,
                   'enterpreneurship': 3,
                   'entrepreneur': 3,
                   'fablab': 1,
                   'future': 1,
                   'industry40': 3,
                   'innovation': 16,
                   'inventor': 2,
                   'laboratory': 2,
                   'machinelearning': 1,
                   'machinist': 1,
                   'maker': 3,
                   'manufacturing': 1,
                   'marketing': 1,
                   'material': 1,
                   'open': 2,
                   'opendesign': 1,
                   'origami': 1,
                   'people': 1,
                   'project': 1,
                   'raspberrypi': 1,
               

'interdisciplinarity': {'co': {'future': 1,
                             'inclusion': 1,
                             'socialinnovation': 1,
                             'sociologist': 1},
                       'f': 1},
'interior': {'co': {'art': 1,
                  'creativity': 1,
                  'design': 1,
                  'designer': 2,
                  'furniture': 1,
                  'innovation': 1,
                  'kids': 1,
                  'textile': 1,
                  'textiles': 1},
            'f': 3},
'internetofthings': {'co': {'3dprint': 1,
                          'arduino': 1,
                          'controlsystems': 1,
                          'ehealth': 1},
                    'f': 1},
'inventor': {'co': {'3dprint': 2,
                  'arduino': 1,
                  'collaborativeeconomy': 1,
                  'craft': 1,
                  'creativity': 1,
                  'cycling': 1,
                  'design': 4,
                  'educatio

                    'maker': 3,
                    'manager': 1,
                    'product': 1,
                    'project': 2,
                    'quality': 1,
                    'research and development': 2,
                    'salesman': 1,
                    'technology': 1},
              'f': 8},
'prova': {'co': {'coriandoli': 1}, 'f': 1},
'python': {'co': {'bigdata': 2,
                'ciao': 1,
                'cooperation': 1,
                'cryptocurrency': 1,
                'css': 1,
                'development': 1,
                'html': 1,
                'innovation': 1,
                'javascript': 1,
                'js': 2,
                'linux': 1,
                'machine learning': 1,
                'opendata': 1,
                'openmaker team member': 1,
                'opensource': 1,
                'raspberrypi': 1,
                'research and development': 1,
                'robot': 1,
                'sass': 1,
                'smart

In [4]:
net['3d']['co']['3dprint']

4

In [5]:
net['3dprint']['co']['3d']

4

### agents.py

In [6]:
# agents.py
from six import string_types

class Agent(object):
    """A generic agent class.

    Args:
        id (:obj:`str`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    Attributes:
        id (:obj:`int`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    """

    def __init__(self, aid, state = 'idle', atype = 'generic'):
        self.id = aid
        self.state = state
        self.type = atype

    def current_state(self):
        """It reports the current state of the agent.

        Returns:
            (:obj:`dict`): A `dict` summarizing the state of the agent.

        """

        state = {'id': self.id, 'state': self.state, 'type': self.type}
        return state
    


class Maker(Agent):
    """A maker (:obj:`Agent`) class.

        Notes:


        Attributes:
            name (:obj:`str`, optional): Name of the maker.
            surname (:obj:`str`, optional): Surname of the maker.
            twitter (:obj:`str`, optional): Twitter name of the maker.
            omid (:obj:`int`, optional): OpenMaker platform id of the maker.
            location (:obj:`str`, optional): The geogrpahic location of the maker.
            entity (:obj:`str`, optional): Whether the agent is one of organization 'O' or person 'P'.
            gender (:obj:`str`, optional): The gender is one of 'M', 'F', and 'X.
            tags (:obj:`set` of :obj:`str`, optional): Self assigned tags of the maker.
            skills (:obj:`set` of :obj:`str`, optional): Skills of the agent.
            technologies(:obj:`set` of :obj:`str`, optional): Technologies used by the agent.
            areas (:obj:`set` of :obj:`str`, optional): Interest areas in terms technologies.
            domains (:obj:`set` of :obj:`str`, optional): Activity domains of the agent.

    """
    name = ''
    surname = ''
    twitter = ''
    entity = ''
    location = ''
    gender = ''
    tags = set()
    skills = set()
    technologies = set()
    areas = set()
    domains = set()

    def __init__(self, aid, state='idle'):
        super().__init__(aid, state, atype = 'Maker')

    def load_profile(self, profile):
        """Updates or loads the profile od the agent.

            Args:
                profile (:obj:`dict`): A python dictionary where keys are attributes of
                this class data attributes and values are the values to be loaded.

            Returns:
                (obj:`bool`): when successful.

        """
        attrs = get_attributes(self)
        attrs.extend(get_attributes(Maker))
        for k,v in profile.items():
            if k not in attrs: continue
            if isinstance(v, string_types): v='"'+str(v)+'"'
            exec("self.{} = {}".format(k,v))

        return True
    

### models.py

#### Expertise    

In [23]:
class CommunityExpertise(object):
    """A python object that contains methods and data models for expertise of a given community.

    Args:
        id (:obj:`str`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    Attributes:
        id (:obj:`int`)): A unique identifier for the agent.
        state (:obj:`str`, optional): Current state of the agent.

    """

    def __init__(self, projects, profiles):
        self.nproj = len(projects)
        self.nprof = len(profiles)
      
        netproj = form_network_relations(projects)
        netprof = form_network_relations(profiles)
        
        self.tags_proj = netproj.keys()
        self.tags_prof = netprof.keys()
        
        self.tf_proj = {t:netproj[t]['f'] for t in netproj.keys()}
        self.tf_prof = {t:netprof[t]['f'] for t in netprof.keys()}
                
        self.mat_proj = {t:netproj[t]['co'] for t in netproj.keys()}
        self.mat_prof = {t:netprof[t]['co'] for t in netprof.keys()}
    
    def _get_cooccurances(self, a, b, tags, mat):
        """A given two skills it returns co-occurance of them.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.
            tags: (:obj:`list` of :obj:`str`): Holds the list of skills(column names),
            mat (:obj:`dict`): A dictionary whose values are dictionary holds
                the frequency counts.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        if a not in tags: return None
        if b not in tags: return None
        if b not in mat[a].keys(): return 0
        
        return mat[a][b]
    
    def get_profile_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in profiles.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_prof,
                                     self.mat_prof) 
    
    def get_project_cooccurances(self, a, b):
        """A given two skills it returns co-occurance of them in projects.

        Args:
            a (:obj:`str`): A skill identifier
            b (:obj:`str`): Another skill identfier.

        Returns:
            None: When either a or b is not occured.
            (:obj:`int`): The raw co-occurance frequency.

        """
        return self._get_cooccurances(a, b,
                                      self.tags_proj,
                                      self.mat_proj) 

    def compute_centralities(self, raw=True):
        """Computes the centralities of skillset that occurance matrix.
        
        Note: 
            This is a mapping of skills on a co-occurance map: the more often a skill is required
            for the projects in a community the more central is the skill to the community.
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the centrality of each skill.
            
        """
        self.centralities = dict()
        for t in self.tags_proj:
            self.centralities[t] = self.get_centrality(t,raw)
        
        return self.centralities
    

    def get_centrality(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from project requirements.
        
        Note: 
            As of the co-occurance map: the more often a skill is required for the projects in a community
            the more central is the skill to the community.
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used whether raw count needs to be returned (default True).

        Returns:
            None: When no project info has found.
            (:obj:`int`): The raw co-occurance frequency when arg raw = True.
            (:obj:`float`): The normalized frequency count as of total project observations.
            
        """
        if a not in self.tags_proj: return 0
        if not self.nproj: return None
        
        fa = self.tf_proj[a]
        if raw: return fa
        return fa / (1.0 * self.nproj)

    def compute_diffusions(self, raw=True):
        """Computes the diffusion of skills among the community members.
        
        Args:
            raw (:obj:`bool`): Is used to determine whether raw count or normalized values should be returned
            (default True).

        Returns:
            (:obj:`dict`): A dictionary holding the diffusion score for each skill.
            
        """
        self.diffusions = dict()
        for t in self.tags_prof:
            self.diffusions[t] = self.get_diffusion(t,raw)
        
        return self.diffusions

    def get_diffusion(self, a, raw=True):
        """Returns the frequency count of a skill in co-occurance matrix formed from agent profiles.

        Note: 
            As of the co-occurance map: the more often a skill is observed within a community the more
            diffused is the skill in the community.
            
        Args:
            a (:obj:`str`): A skill identifier
            raw (:obj:`bool`): Is used to determine whether raw count needs to be returned (default True).

        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of total profiles/members in the community.
            
        """
        
        if a not in self.tags_prof: return 0
        if not self.nprof: return None
        
        fa = self.tf_prof[a]
        if raw: return fa
        return fa / (1.0 * self.nprof)

    def compute_distinctivenesses(self):
        """Computes the distinctiveness of skills within the community parctices.
        Returns:
            (:obj:`dict`): A dictionary holding the distinctiveness score for each skill.
            
        """
        self.distinctivenesses = dict()
        for t in self.tags_proj:
            self.distinctivenesses[t] = self.get_distinctiveness(t)
        
        return self.distinctivenesses
    
    def get_distinctiveness(self, a):
        """Returns a score to measure to what extend an important skill for the community is owned by less
            members.
            
        Args:
            a (:obj:`str`): A skill identifier
        Returns:
            None: When no profile info has found.
            (:obj:`int`): The raw co-occurance frequency when the arg raw = True.
            (:obj:`float`): The normalized frequency count as of total profiles/members in the community.
            
        """
        x = self.get_diffusion(a)
        y = self.get_centrality(a)
        if not x: return None
        if y is None: return None  
        return y / x


    def get_union(a,b):
        """Returns total occurances where a or b is observed.

        """
        fa = get_project_cooccurances(a,a)
        fb = get_project_cooccurances(b,b)
        fab = get_project_cooccurances(a,b)
        return fa + fb - fab

    def get_difference(a, b, raw=True, exclusive=True):
        """Returns the count of occurances of a without b.

        """
        fa = get_project_cooccurances(a,a)
        fab = get_project_cooccurances(a,b)
        dif = fa - fab
        if raw: return dif
        if exclusive: return dif / (1.0 * fa)
        return dif / (1.0 * get_union(a,b))

    def get_similarity(a,b, raw=True, exclusive=True):
        """Returns the count of co-occurances of a and b.

        Note: When the union is used as the the reference, that is when exclusive is false,
            sim(a,b) = sim(b,a)

        """
        fab = get_project_cooccurances(a,b)
        if raw: return fab
        fa = get_project_cooccurances(a,a)
        if exclusive: return fab / (1.0 * fa)
        return fab / (1.0 * get_union(a,b))

    def get_resemblance(a,b):
        """Returns the resemblance of a and b in terms of both their similarities and differences.

        """
        fab = get_project_cooccurances(a,b)
        dif_ab = get_difference(a,b,raw=False)
        dif_ba = get_difference(b,a,raw=False)
        sim_ab = get_similarity(a,b,raw=False,exclusive=False)
        return sim_ab * (1 - abs(dif_ab - dif_ba))

    def get_compatibility(a,b):
        """Returns a score to measure: given all occurances of skill b to what extend 
            skill a cooccurs with skill b.

        Note: If, for instance, skill b happens to be occuring only when skill a occurs
            then the compatibility of a to b would be 1. In a asymmetric compatibility (a to b VS b to a)
            where one of the compatibilities is almost 1 or 1, this may point out a  skill and sub-skill 
            relation.

        """
        return get_similarity(b,a,raw=False,exclusive=True)

In [24]:
coMat = load_skill_cooccurance()
CE = CommunityExpertise(coMat,coMat)
#pp.pprint(CE.mat_proj)

In [25]:
CE.compute_distinctivenesses()

{'3d': 1.0,
 '3dmodel': 1.0,
 '3dp': 1.0,
 '3dprint': 1.0,
 '3dscanning': 1.0,
 'accessibility': 1.0,
 'accessibilità': 1.0,
 'additive': 1.0,
 'aec': 1.0,
 'aeronautics': 1.0,
 'aerospace': 1.0,
 'agile': 1.0,
 'agrotourism': 1.0,
 'ai': 1.0,
 'airbrush': 1.0,
 'anthropocene': 1.0,
 'applied': 1.0,
 'aquaponics': 1.0,
 'ar': 1.0,
 'architechture': 1.0,
 'architect': 1.0,
 'arduino': 1.0,
 'art': 1.0,
 'arte': 1.0,
 'artist': 1.0,
 'audio': 1.0,
 'automation': 1.0,
 'automotive': 1.0,
 'awareness': 1.0,
 'beauty': 1.0,
 'bellezza': 1.0,
 'bettereurope': 1.0,
 'biesse': 1.0,
 'bigdata': 1.0,
 'bim': 1.0,
 'biohacker': 1.0,
 'biological': 1.0,
 'biology': 1.0,
 'board': 1.0,
 'bottle': 1.0,
 'building': 1.0,
 'business': 1.0,
 'business strategy': 1.0,
 'bussines': 1.0,
 'cad': 1.0,
 'ceo': 1.0,
 'ceramics': 1.0,
 'cfd': 1.0,
 'changemaker': 1.0,
 'chemisty': 1.0,
 'childrens': 1.0,
 'ciao': 1.0,
 'cities': 1.0,
 'citizens': 1.0,
 'cloud': 1.0,
 'cnc': 1.0,
 'cncrouter': 1.0,
 'co design

In [ ]:
CE.compute_diffusions(raw = False)

### interfaces.py

In [ ]:
abs(2 - 3)

# Tests

In [ ]:
m = Maker(1)

In [ ]:
m.load_profile({'name': 'bulent'})

In [ ]:
m.name

In [ ]:
a = exec("'s'")
print(a)

In [ ]:
MEMBER_PROFILES = load_member_profiles(fname = './data/crm_profile.pickle')

In [ ]:
aprofile = get_a_dict_item(MEMBER_PROFILES)
aid = list(aprofile.keys())[0]
profile = aprofile[aid]
profile['id'] = aid
pp.pprint(profile)

In [ ]:
m.load_profile(profile)

In [ ]:
type(MEMBER_PROFILES)

In [ ]:
print(m.gender)